<a href="https://colab.research.google.com/github/nirb28/ee-predict/blob/main/cd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/ee-predict/' + relativePath
    else:
        return relativePath
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np, os, csv
import warnings
warnings.filterwarnings('ignore')

In [27]:
MODEL_REFRESH_STRATEGY = "RETRAIN" # Retrain model after every loop: RETRAIN, PRETRAIN
OPTIMIZE_STRATEGY = "NEWLY_ADDED" # Only optimize the newly added catalyst: NEWLY_ADDED, ALL
DEFAULT_PCA_DIMENSION = 4
START_SAMPLE_SIZE = 3
DATASET = "LARGE" # "LARGE" or "REDUCED"
if DATASET == "REDUCED":
  DATAFILE_NAME = "reduced_dim_space_ddG.csv"
  X_COLS = ['x', 'y', 'z']
else:
  DATAFILE_NAME = "merged_large_catalyst.csv"
  with open(resolve_path_gdrive("models/high_corr_cols.txt")) as f:
    reader = csv.reader(f)
    X_COLS = list(reader)[0][:-1]

df = pd.read_csv(resolve_path_gdrive('data/' + DATAFILE_NAME))
## removing ddG = 0 and indexing catalysts
df = df[df['ddG'] != 0]
df.set_index('Catalyst', inplace=True)

In [28]:
# method to make a regression model from the provided catalysts
from sklearn.cross_decomposition import PLSRegression
def make_pls_model(df_catalysts, PCA_DIMENSION=DEFAULT_PCA_DIMENSION):
    X = df_catalysts[X_COLS]
    y = df_catalysts['ddG']
    model = PLSRegression(n_components=PCA_DIMENSION)
    print("PLS: Fitting model with {} rows and {} columns. PCA dimensions are {}"
        .format(len(X), len(X.columns), PCA_DIMENSION))
    # fitting the model
    return model.fit(X, y)

## Importing model created in "eda.ipynb"
from joblib import dump, load
def load_model(path):
  return load(path)

pls_large_saved_model = load_model(resolve_path_gdrive('models/pls_large.joblib'))
print(pls_large_saved_model)
## Read the dimension directly from the model so that if the model changes
## the dimension will be reflected automatically
PCA_DIMENSION = pls_large_saved_model.n_components

def pls_predict_ee(pls_model, properties):
    return pls_model.predict(properties.reshape(1,-1))

## Add a predicted ddG column to the df
df['pred_ddG'] = df.apply(lambda x: pls_predict_ee(pls_large_saved_model, np.array(x[X_COLS]))[0][0], axis=1)

PLSRegression(n_components=4)


In [29]:
# start with a few randomly selected catalyst
random_catalyst_df = df.sample(START_SAMPLE_SIZE)
## This will be used to track progress of coordinate descent
print("The max value (target) of the ddG is at {}. Currently we have reached {}.".format(df['ddG'].max(), random_catalyst_df['ddG'].max()))
random_catalyst_df

The max value (target) of the ddG is at 1.640249758. Currently we have reached 0.187525382.


,1,2,3,4,5,6,7,8,9,10,...,3967,3968,3969,3970,3971,3972,3973,3974,ddG,pred_ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
13_10_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.4717,-0.76848,-0.178536,-0.29859,-0.23864,-0.27751,-0.230633,-0.672194,0.187525,0.159338
7_3_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.4717,-0.76848,-0.178536,-0.29859,-0.23864,-0.27751,-0.230633,-0.672194,0.055715,0.196258
7_7_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.4717,-0.76848,-0.178536,-0.29859,-0.23864,-0.27751,-0.230633,-0.672194,0.004738,0.446809


In [30]:
original_ee = df['ddG']
# Function to optimize catalyst properties using coordinate descent
def optimize_catalysts(catalysts, pls_model=None, iterations=5, cd_iterations=2, step_size=0.001):
    if pls_model == None:
        pls_model = make_pls_model(random_catalyst_df)
    optimized_catalysts = np.copy(catalysts)
    for _ in range(iterations):
        for i in range(len(optimized_catalysts)):
            ## we have few options for the original_ee. We can use from the dataset or get the prediction
            ## from the model. The prediction makes sense as we want to traverse in the direction where
            ## ee improves according to the model

            ## when using ddG from dataset
            #original_ee = df.iloc[df.index.get_loc(catalysts[i:i+1].index[0])]['ddG']
            ## when using the model
            original_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
            for x in range(len(optimized_catalysts[i])):
                for cd in range(cd_iterations):
                    old_value = optimized_catalysts[i, x]
                    optimized_catalysts[i, x] = old_value + step_size
                    new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value - step_size
                        new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value
                        break
                    #print('Found a direction for higher ddG')
    return optimized_catalysts


In [31]:
if MODEL_REFRESH_STRATEGY == "RETRAIN":
  new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS])
else:
  new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)

PLS: Fitting model with 3 rows and 62 columns. PCA dimensions are 4


In [32]:
# Perform k-nearest neighbors analysis
def recomputeKNN(df, removePointsDF = None):
  if len(removePointsDF) > 0:
    df.drop(removePointsDF.index, axis=0, inplace=True)
  num_neighbors = 1  # Number of neighbors to consider
  knn = NearestNeighbors(n_neighbors=num_neighbors)
  print('KNN: Fitting model with {} points.'.format(len(df)))
  knn.fit(df)  # Using all the catalysts
  return knn

knn = recomputeKNN(df[X_COLS], random_catalyst_df)

KNN: Fitting model with 316 points.


In [33]:
## Recursive Loop
## The method will get unique neighbors that are not already in the list.
## If the 1st neighbor is already in the list then we try the 2nd, 3rd and so on
## till a unique neighbor is found
def find_unique_neighbors_add(knn, point, addToDf, neighbor_number=1):
    distances, indices = knn.kneighbors([point], neighbor_number)
    neighbor_catalyst = df.iloc[[indices[0][neighbor_number-1]]]
    if neighbor_catalyst.index[0] in addToDf.index:
        #get next neighbor
        neighbor_catalyst = find_unique_neighbors_add(knn, point, addToDf, neighbor_number+1)
    return neighbor_catalyst

def process_neighbors(knn, for_points, addToDf):
    for i in range(len(for_points)):
        neighbor_catalyst = find_unique_neighbors_add(knn, for_points[i], addToDf)
        ## Add found catalysts into our batch (random_catalyst_df)
        if len(neighbor_catalyst) > 0:
            addToDf = pd.concat([addToDf, neighbor_catalyst], ignore_index=False, sort=True)
    return addToDf

random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
random_catalyst_df[X_COLS]

,906,907,908,922,923,937,950,951,952,1113,...,3173,3524,3525,3547,3548,3628,3688,3916,3940,3951
Catalyst,,,,,,,,,,,,,,,,,,,,,
13_10_1,0.000000,0.253968,0.365079,0.190476,0.380952,0.460317,0.063492,0.301587,0.539683,0.158730,...,0.000381,-0.033238,-0.032952,0.000000,0.000000,-0.019048,-0.033333,-8.810000e-19,0.000000,0.000000
7_3_1,0.333333,0.333333,0.266667,0.333333,0.266667,0.400000,0.333333,0.333333,0.200000,0.333333,...,0.050000,-0.055000,-0.073800,-0.040000,-0.040000,-0.090000,-0.015000,-1.920000e-02,0.020000,0.000000
7_7_1,0.000000,0.000000,0.000000,0.000000,0.076271,0.059322,0.000000,0.000000,0.016949,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
22_1_4,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.090909,0.090909,0.000000,...,-0.041273,0.054545,0.068182,0.027273,0.068182,0.000000,0.054545,-1.483636e-01,-0.068182,0.013636
7_2_4,0.000000,0.041379,0.103448,0.124138,0.089655,0.117241,0.020690,0.179310,0.234483,0.193103,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
7_4_4,0.000000,0.043478,0.050725,0.043478,0.101449,0.101449,0.000000,0.036232,0.043478,0.195652,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000


In [34]:
# We will now do all this in a single loop

## Maximum of entire dataset
max_ddG = df['ddG'].max()
print("The max value (target) of the ddG is {}. Model training strategy is {}".format(max_ddG, MODEL_REFRESH_STRATEGY))

## Get random rows
#random_catalyst_df = df.loc[not_highest_index].sample(START_SAMPLE_SIZE)
## Get lowest ddG to start
#random_catalyst_df = df.sort_values('ddG').head(START_SAMPLE_SIZE)
## Static list to start
random_catalyst_df = df[df['ddG'] > .5].sort_values('ddG').head(START_SAMPLE_SIZE)

while(len(random_catalyst_df) < len(df)):
    print("The current maximum ddg is at: " + str(random_catalyst_df['pred_ddG'].max()))
    what_to_optimize = random_catalyst_df[X_COLS]
    if OPTIMIZE_STRATEGY == "NEWLY_ADDED":
        what_to_optimize = what_to_optimize.tail(START_SAMPLE_SIZE)
    new_optimized_catalysts = optimize_catalysts(what_to_optimize,
        pls_large_saved_model if MODEL_REFRESH_STRATEGY == "RETRAIN" else None)
    random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
    # recompute knn so that we dont find the same neighbors in the next loop around
    knn = recomputeKNN(df[X_COLS], random_catalyst_df)
    if random_catalyst_df['ddG'].max() >= max_ddG:
        print("Found the highest ddG {}. Exiting.".format(str(random_catalyst_df['ddG'].max())))
        break

The max value (target) of the ddG is 1.640249758. Model training strategy is RETRAIN
The current maximum ddg is at: 0.5210952334395327
KNN: Fitting model with 313 points.
The current maximum ddg is at: 1.0940875029169244
KNN: Fitting model with 310 points.
The current maximum ddg is at: 1.0940875029169244
KNN: Fitting model with 307 points.
The current maximum ddg is at: 1.0940875029169244
KNN: Fitting model with 304 points.
The current maximum ddg is at: 1.1740024154135276
KNN: Fitting model with 301 points.
The current maximum ddg is at: 1.2280844168320835
KNN: Fitting model with 298 points.
The current maximum ddg is at: 1.2280844168320835
KNN: Fitting model with 295 points.
The current maximum ddg is at: 1.2280844168320835
KNN: Fitting model with 292 points.
The current maximum ddg is at: 1.2280844168320835
KNN: Fitting model with 289 points.
The current maximum ddg is at: 1.2280844168320835
KNN: Fitting model with 286 points.
The current maximum ddg is at: 1.4036967286687805
KNN: 

In [ ]:
# sample prediction with model
properties1 = (np.array([ 0.75813041,  0.67727654,  0.68692902,  0.57403813, -0.39901283,
       -0.83317389, -0.69366434, -0.76423736, -0.18775295, -0.89697628,
       -0.93897206, -0.64303136, -0.50508299, -0.82567658, -0.29773217,
       -0.23605932, -0.26722481,  0.02879822, -0.69189841]))
#pls_predict_ee(pls_large_saved_model, properties1)
# random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]

In [22]:
# prompt: get last 3 rows of random_catalyst_df[X_COLS]

random_catalyst_df[X_COLS].tail(3)


,906,907,908,922,923,937,950,951,952,1113,...,3173,3524,3525,3547,3548,3628,3688,3916,3940,3951
Catalyst,,,,,,,,,,,,,,,,,,,,,
1_4_4,0.000,0.039604,0.049505,0.039604,0.099010,0.089109,0.000000,0.009901,0.059406,0.267327,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0
11_10_4,0.000,0.000000,0.076923,0.000000,0.153846,0.076923,0.076923,0.153846,0.230769,0.076923,...,-0.011538,-0.034615,-0.034154,0.000000,0.000000,-0.023077,-0.034615,-0.023077,0.0000,0.0
15_6_2,0.000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.090909,0.000000,...,-0.013636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0
2_3_2,0.375,0.375000,0.375000,0.375000,0.500000,0.375000,0.375000,0.375000,0.375000,0.375000,...,0.056250,0.018750,0.039000,0.000000,0.000000,-0.093750,0.018750,0.001500,0.0375,0.0
21_1_2,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.000000,...,-0.028800,0.030000,0.060000,0.030000,0.030000,0.000000,0.060000,-0.148800,-0.0600,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11_7_1,0.000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.066667,...,-0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0
23_7_1,0.000,0.000000,0.000000,0.075758,0.348485,0.257576,0.212121,0.257576,0.287879,0.015152,...,-0.013864,-0.002273,0.000000,-0.006818,-0.006818,0.000000,0.000000,0.000000,0.0000,0.0
21_7_4,0.000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.133333,0.133333,0.066667,...,-0.019200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0
